# Análsis de carpetas de investigación del 2023

Estos datos corresponden al periodo del 01/Enero/2023 al 31/Marzo/2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.impute import KNNImputer
import math
import re

## Entendimiento de los datos

In [2]:
df = pd.read_csv('data/da_2023.csv')
df.shape

(64344, 22)

In [3]:
df.columns

Index(['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Año_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'alcaldia_hechos', 'municipio_hechos', 'colonia_datos',
       'fgj_colonia_registro', 'latitud', 'longitud'],
      dtype='object')

In [4]:
df.head(15)

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,9361878,2023,Enero,2023-01-01,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,NaN,NaN,MORAL,OFENDIDO,...,Diciembre,2022-12-31,22:08:00,00:09:00,COYOACAN,NaN,INSURGENTES CUICUILCO,INSURGENTES CUICUILCO,19.305532,-99.186308
1,9361879,2023,Enero,2023-01-01,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,50.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,23:50:00,00:41:00,VENUSTIANO CARRANZA,NaN,MOCTEZUMA 2A SECCION IV,MOCTEZUMA 2A SECCIÓN,19.429797,-99.098672
2,9361880,2023,Enero,2023-01-01,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,34.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,20:30:00,00:44:00,GUSTAVO A. MADERO,NaN,SAN JUAN DE ARAGON 3A SECCION (U HAB) I,SAN JUAN DE ARAGÓN III SECCIÓN,19.451641,-99.076529
3,9361881,2023,Enero,2023-01-01,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,Masculino,NaN,FISICA,LESIONADO,...,Diciembre,2022-12-31,23:30:00,00:47:00,COYOACAN,NaN,XOTEPINGO,XOTEPINGO,19.323245,-99.138016
4,9361882,2023,Enero,2023-01-01,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,Masculino,90.0,FISICA,CADAVER,...,Diciembre,2022-12-31,10:30:00,00:56:00,GUSTAVO A. MADERO,NaN,NaN,NaN,NaN,NaN
5,9361883,2023,Enero,2023-01-01,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Masculino,NaN,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,23:15:00,01:04:00,IZTAPALAPA,NaN,PASEOS DE CHURUBUSCO,CENTRAL DE ABASTO,19.377017,-99.091991
6,9361885,2023,Enero,2023-01-01,DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO V...,DELITO DE BAJO IMPACTO,Masculino,58.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,22:47:00,01:33:00,IZTAPALAPA,NaN,CULHUACAN (PBLO),PUEBLO CULHUACÁN,19.338649,-99.108793
7,9361888,2023,Enero,2023-01-01,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,32.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,23:00:00,01:43:00,MIGUEL HIDALGO,NaN,PENSIL SUR,PENSIL SUR,19.445881,-99.195806
8,9361889,2023,Enero,2023-01-01,PERSONAS EXTRAVIADAS Y AUSENTES,DELITO DE BAJO IMPACTO,Masculino,14.0,FISICA,VICTIMA,...,Diciembre,2022-12-31,16:00:00,01:48:00,IZTAPALAPA,NaN,STA MA AZTAHUACAN (PBLO),SANTA MARIA AZTAHUACAN,19.347793,-99.019130
9,9361892,2023,Enero,2023-01-01,DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO V...,DELITO DE BAJO IMPACTO,Masculino,31.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2022-12-31,23:50:00,01:55:00,CUAUHTEMOC,NaN,PERALVILLO I,PERALVILLO,19.461487,-99.137570


In [5]:
df.dtypes

idCarpeta                 int64
Año_inicio                int64
Mes_inicio               object
FechaInicio              object
Delito                   object
Categoria                object
Sexo                     object
Edad                    float64
TipoPersona              object
CalidadJuridica          object
competencia              object
Año_hecho               float64
Mes_hecho                object
FechaHecho               object
HoraHecho                object
HoraInicio               object
alcaldia_hechos          object
municipio_hechos         object
colonia_datos            object
fgj_colonia_registro     object
latitud                 float64
longitud                float64
dtype: object

In [6]:
df.nunique()

idCarpeta               61333
Año_inicio                  1
Mes_inicio                  3
FechaInicio                90
Delito                    255
Categoria                  16
Sexo                        2
Edad                      100
TipoPersona                 2
CalidadJuridica             5
competencia                 3
Año_hecho                  39
Mes_hecho                  12
FechaHecho               1434
HoraHecho                1441
HoraInicio               1455
alcaldia_hechos            18
municipio_hechos           91
colonia_datos            1655
fgj_colonia_registro     1538
latitud                 57654
longitud                57653
dtype: int64

In [7]:
df["Año_hecho"].describe()

count    64337.000000
mean      2022.731616
std          1.271916
min       1962.000000
25%       2023.000000
50%       2023.000000
75%       2023.000000
max       2023.000000
Name: Año_hecho, dtype: float64

### Tenemos delitos cometidos desde el año **1962** a los cuáles se les abrió carpetas de investigación hasta el año **2023**

In [8]:
df["Año_hecho"].unique()

array([2022., 2023., 2021., 2018., 2019., 2020., 2013., 1998., 2007.,
       2017., 2014., 2015., 2010., 2011.,   nan, 2003., 2016., 2005.,
       2000., 2006., 1993., 1995., 2012., 2002., 2008., 1969., 1987.,
       2009., 1967., 1990., 1999., 2004., 1994., 1974., 1976., 1985.,
       1962., 1991., 1972., 2001.])

In [9]:
df["Categoria"].unique()

array(['DELITO DE BAJO IMPACTO',
       'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA',
       'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA', 'HECHO NO DELICTIVO',
       'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', 'VIOLACIÓN',
       'ROBO A NEGOCIO CON VIOLENCIA',
       'ROBO A REPARTIDOR CON Y SIN VIOLENCIA', 'HOMICIDIO DOLOSO',
       'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA',
       'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA',
       'ROBO A CASA HABITACIÓN CON VIOLENCIA',
       'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA', 'SECUESTRO'],
      dtype=object)

In [10]:
df.loc[df["Año_hecho"]==1962]

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
44757,9408996,2023,Marzo,2023-03-06,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,60.0,FISICA,VICTIMA Y DENUNCIANTE,...,Septiembre,1962-09-11,13:45:00,18:39:00,COYOACAN,NaN,DEL CARMEN,DEL CARMEN,19.351477,-99.158951


In [11]:
df["competencia"].unique()

array(['FUERO COMUN', 'HECHOS NO DELICTIVOS', 'INCOMPETENCIAS'],
      dtype=object)

### Resumen del entendimiento de los datos

1. Existen carpetas de investigación sobre delitos cometidos desde 1962, abiertas hasta el 2023.

2. Tenemos **16 categorías de delitos**.

3. Hay **5 tipos de calidades jurídicas**.

4. Describiendo los **3 competencias de delitos**:
    
    * Fuero común: Aquellos delitos que afectas directamente a la persona.
    * Hechos no delictivos: No son delitos.
    * Incompetencias: La persona es incapaz de ser juzgada o de ser responsable de sus acciones debido a condiciones físicas o mentales.
    
    
    
5. En la columna de alcaldías estamos considerando un **extra llamado 'fuera de la ciudad de méxico'**

6. Tenemos **255 tipos de delitos**.

## Limpieza

1. [X] Eliminar columnas que se relacionen con el tiempo en que se hizo la denuncia
2. [X] Estandarizar a minusculas las columnas restantes
3. [X] Renombrar columnas a 1 sola palabra
4. [X] Eliminar la palabra hecho de todas las columnas que la contengan
5. [X] Convertir a formato de fecha si corresponde
6. [X] Eliminar la columna de Municipio ya que solo vamos a analizar dentro de CDMX
7. [ ] Rellenar los NaN  

        7.1 [ ] 'sexo' no, hasta tener separados los dataframes  
        
        7.2 [ ] 'edad' no, hasta tener separados los dataframes  
        
        7.3 [X] 'persona' no, eliminar los registros que no tengan este campo  
        
        7.4 [X] 'año, mes, fecha y hora' no, son 7 entonces eliminemos  
        
        7.5 [ ] 'colonia' si, quizas teniendo en cuenta las colonias que mas se repiten por alcaldia  
        
        7.6 [X] 'latitud y longitud' borrar columnas (realmente no se usarán)
        
8. [X] Cambiar a enteros los años
9. [X] Si existe un tipo de dato de "hora", cambiarlo en la que corresponda  

        9.1 [X] Eliminar aquellos registros con categoria "hecho no delictivo" y calidad "incompetencias"
        
10. [X] Separar en 2 DataFrames teniendo en cuenta el tipo de persona
11. [ ] En ambos dataframes mantener aquellos delitos con competencia de **fuero común e incompetencias (no estoy seguro del ultimo)**
12. [X] Del dataframe de personas morales, eliminar las columnas de sexo, edad, tipopersona
13. [ ] En la columna de edades de personas físicas, llenar los NA con algún **algoritmo de imputación**


... Si hay más, apúntelos aquí

### Paso 1

In [12]:
df.drop(
        columns=["fgj_colonia_registro", "idCarpeta", "Año_inicio", "Mes_inicio", "FechaInicio", "HoraInicio"], 
        inplace=True
        )

### Paso 2

In [13]:
df.rename(columns= lambda x: x.lower(), inplace=True)

### Paso 3

In [14]:
df.rename(columns = 
         {
             'tipopersona': 'persona',
             'calidadjuridica': 'calidad',
             'colonia_datos': 'colonia'
         }, 
        inplace=True)

### Paso 4

In [15]:
def find(name):
    return re.sub(r'_?hechos?$', '', name)

df.columns = df.columns.to_frame()[0].agg(find)

### Paso 5

### Paso 6: Eliminar Municipio

In [16]:
df.drop(columns="municipio", inplace=True)

### Paso 7.3, 7.4

In [17]:
# Borrar persona, año, mes fecha y hora nulos
df.dropna(subset=["persona", "año", "mes", "fecha", "hora"], inplace=True)

### Paso 7.5

### Paso 7.6

In [18]:
df.drop(columns=["latitud", "longitud"], inplace=True)

In [19]:
pd.to_datetime(df.fecha)

0       2022-12-31
1       2022-12-31
2       2022-12-31
3       2022-12-31
4       2022-12-31
           ...    
64339   2023-03-31
64340   2023-03-30
64341   2023-03-31
64342   2023-03-31
64343   2023-03-31
Name: fecha, Length: 64205, dtype: datetime64[ns]

### Paso 8

In [20]:
df['año'].astype(int)

0        2022
1        2022
2        2022
3        2022
4        2022
         ... 
64339    2023
64340    2023
64341    2023
64342    2023
64343    2023
Name: año, Length: 64205, dtype: int64

### Paso 9

In [21]:
pd.to_datetime(df['hora']).dt.time

0        22:08:00
1        23:50:00
2        20:30:00
3        23:30:00
4        10:30:00
           ...   
64339    07:30:00
64340    22:00:00
64341    07:00:00
64342    10:01:00
64343    17:30:00
Name: hora, Length: 64205, dtype: object

### Paso 9.1

In [22]:
df = df[np.logical_and(df["categoria"] != "HECHO NO DELICTIVO", df["calidad"] != "INCOMPETENCIAS")]

### Paso 10: separacion de dataframe

In [23]:
df_fisica = df.query('persona == "FISICA"')
df_moral = df.query('persona == "MORAL"')

### Paso 11

### Paso 12, de las morales eliminar sexo y edad

In [24]:
df_moral.drop(labels=["sexo", "edad"], inplace=True)

KeyError: "['sexo', 'edad'] not found in axis"

### Paso 13, rellenar edades en df_fisica

In [25]:
#categorias = df_fisica.categoria.unique()
#dic_cat = dict()
#i = 0
#for categoria in categorias:
#    dic_cat[categoria] = i
#   i += 1

#dic_cat
df_fisica.info()
df_fisica.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53902 entries, 1 to 64343
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   delito       53902 non-null  object 
 1   categoria    53902 non-null  object 
 2   sexo         53902 non-null  object 
 3   edad         44165 non-null  float64
 4   persona      53902 non-null  object 
 5   calidad      53902 non-null  object 
 6   competencia  53902 non-null  object 
 7   año          53902 non-null  float64
 8   mes          53902 non-null  object 
 9   fecha        53902 non-null  object 
 10  hora         53902 non-null  object 
 11  alcaldia     53902 non-null  object 
 12  colonia      50718 non-null  object 
dtypes: float64(2), object(11)
memory usage: 5.8+ MB


,delito,categoria,sexo,edad,persona,calidad,competencia,año,mes,fecha,hora,alcaldia,colonia
1,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,50.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2022.0,Diciembre,2022-12-31,23:50:00,VENUSTIANO CARRANZA,MOCTEZUMA 2A SECCION IV
2,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,34.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2022.0,Diciembre,2022-12-31,20:30:00,GUSTAVO A. MADERO,SAN JUAN DE ARAGON 3A SECCION (U HAB) I
3,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,Masculino,NaN,FISICA,LESIONADO,FUERO COMUN,2022.0,Diciembre,2022-12-31,23:30:00,COYOACAN,XOTEPINGO
5,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Masculino,NaN,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2022.0,Diciembre,2022-12-31,23:15:00,IZTAPALAPA,PASEOS DE CHURUBUSCO
6,DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO V...,DELITO DE BAJO IMPACTO,Masculino,58.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2022.0,Diciembre,2022-12-31,22:47:00,IZTAPALAPA,CULHUACAN (PBLO)


In [ ]:
df_fisica.isna().sum()
#df_moral.isna().sum()